<a href="https://colab.research.google.com/github/khairooo/Quality_Engineering_app/blob/main/quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages :
import pandas as pd
import numpy as np
import random


#  a Dataset to test the project

In [ ]:
import pandas as pd
import numpy as np
import random

# Set the seed for reproducibility
np.random.seed(42)

# Number of rows
num_rows = 200

# Generate timestamps
timestamps = pd.date_range(start='1/1/2023 00:00:00', periods=num_rows, freq='S')

# Generate HotDiam (Outer Diameters)
hot_diam = np.random.uniform(100, 200, num_rows)

# Generate totalMinWall (Wall Thicknesses)
total_min_wall = np.random.uniform(5, 15, num_rows)

# Create a DataFrame
data = {
    'TimeStamp': timestamps,
    'HotDiam': hot_diam,
    'totalMinWall': total_min_wall
}
df = pd.DataFrame(data)

# Introduce dashes in 10% of the rows
num_dashes = int(num_rows * 0.1)
dash_indices = random.sample(range(num_rows), num_dashes)

for idx in dash_indices:
    df.loc[idx, 'HotDiam'] = '-'
    df.loc[idx, 'totalMinWall'] = '-'

# Convert columns back to float where possible, leaving dashes as strings
df['HotDiam'] = pd.to_numeric(df['HotDiam'], errors='coerce')
df['totalMinWall'] = pd.to_numeric(df['totalMinWall'], errors='coerce')

# Display the DataFrame
print(df)

# Save the DataFrame to an Excel file
df.to_excel('gas_pipe_data.xlsx', index=True)


<ipython-input-2-f6bc05090e7d>:12: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  timestamps = pd.date_range(start='1/1/2023 00:00:00', periods=num_rows, freq='S')
<ipython-input-2-f6bc05090e7d>:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'HotDiam'] = '-'
<ipython-input-2-f6bc05090e7d>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'totalMinWall'] = '-'


              TimeStamp     HotDiam  totalMinWall
0   2023-01-01 00:00:00  137.454012     11.420316
1   2023-01-01 00:00:01  195.071431      5.841400
2   2023-01-01 00:00:02  173.199394      6.616287
3   2023-01-01 00:00:03  159.865848     13.985542
4   2023-01-01 00:00:04  115.601864     11.064291
..                  ...         ...           ...
195 2023-01-01 00:03:15  134.920957     14.307573
196 2023-01-01 00:03:16  172.595568     13.584128
197 2023-01-01 00:03:17  189.711026      9.289940
198 2023-01-01 00:03:18  188.708642     12.508711
199 2023-01-01 00:03:19         NaN           NaN

[200 rows x 3 columns]


In [ ]:
# Load the Excel file
file_path = '/content/gas_pipe_data.xlsx'
df = pd.read_excel(file_path)

In [ ]:
df.head()

,Unnamed: 0,TimeStamp,HotDiam,totalMinWall
0,0,2023-01-01 00:00:00,137.454012,11.420316
1,1,2023-01-01 00:00:01,195.071431,5.841400
2,2,2023-01-01 00:00:02,173.199394,6.616287
3,3,2023-01-01 00:00:03,159.865848,13.985542
4,4,2023-01-01 00:00:04,115.601864,11.064291


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1OidAB9y2q2IlDOImO5NkRZJ-wWcD8hWZTk74TtXKqBI#gid=0


In [ ]:
# data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    200 non-null    int64         
 1   TimeStamp     200 non-null    datetime64[ns]
 2   HotDiam       180 non-null    float64       
 3   totalMinWall  180 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 6.4 KB


In [ ]:
describe = df.describe(exclude=[np.datetime64])
describe

,Unnamed: 0,HotDiam,totalMinWall
count,200.000000,180.000000,180.000000
mean,99.500000,148.819213,10.032261
std,57.879185,29.827693,2.920846
min,0.000000,100.552212,5.091971
25%,49.750000,122.858241,7.614625
50%,99.500000,149.996376,10.359949
75%,149.250000,176.099369,12.475222
max,199.000000,198.688694,14.905051


# Control chart for wall thickness and OD vs Times

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_stacked_control_charts(data):
    """
    Creates two control charts stacked vertically: one for HotDiam and one for totalMinWall.

    Parameters:
    data (pd.DataFrame): Input dataframe with TimeStamp, HotDiam, and totalMinWall columns.
    """

    # Create a figure with two subplots arranged vertically
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)

    # --- HotDiam Control Chart (Top Subplot) ---
    # Calculate control limits
    mean_hd = pd.to_numeric(df['HotDiam'], errors='coerce').mean()
    std_hd = pd.to_numeric(df['HotDiam'], errors='coerce').std()
    ucl_hd = 180
    lcl_hd = 120

    # Add traces to the top subplot
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=data['HotDiam'], mode='lines+markers', name='OD', line=dict(color='blue')),
        row=1, col=1  # Specify top subplot
    )
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[mean_hd] * len(data), mode='lines', name='Mean', line=dict(color='green', dash='dash')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[ucl_hd] * len(data), mode='lines', name='UCL', line=dict(color='red', dash='dash')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[lcl_hd] * len(data), mode='lines', name='LCL', line=dict(color='red', dash='dash')),
        row=1, col=1
    )

    # --- totalMinWall Control Chart (Bottom Subplot) ---
    # Calculate control limits (using similar logic as for HotDiam)
    mean_wall = pd.to_numeric(df['totalMinWall'], errors='coerce').mean()
    std_wall = pd.to_numeric(df['totalMinWall'], errors='coerce').std()
    # Adjust UCL and LCL for totalMinWall as needed
    ucl_wall = mean_wall + 3 * std_wall
    lcl_wall = mean_wall - 3 * std_wall

    # Add traces to the bottom subplot
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=data['totalMinWall'], mode='lines+markers', name='Wall Thickness', line=dict(color='purple')),
        row=2, col=1  # Specify bottom subplot
    )
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[mean_wall] * len(data), mode='lines', name='Mean', line=dict(color='green', dash='dash')),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[ucl_wall] * len(data), mode='lines', name='UCL', line=dict(color='Red', dash='dash')),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[lcl_wall] * len(data), mode='lines', name='LCL', line=dict(color='red', dash='dash')),
        row=2, col=1
    )

    # Update layout for the entire figure
    fig.update_layout(
        title='Control Charts for OD and Wall Thickness over Time',
        xaxis_title='Time',
        height=800,  # Increase height to accommodate two charts
        showlegend=True,
        hovermode='x unified'
    )

    # Configure x-axis (shared by both subplots)
    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    # Update y-axis titles for each subplot
    fig.update_yaxes(title_text="OD", row=1, col=1)
    fig.update_yaxes(title_text="Wall Thickness", row=2, col=1)

    return fig

# Example usage:
fig = create_stacked_control_charts(df)
fig.show()

In [ ]:
# Calculate min and max for each column, excluding dashes and converting to numeric
min_wall = pd.to_numeric(df['totalMinWall'], errors='coerce').min()
min_avg_wall = pd.to_numeric(df['totalMinWall'], errors='coerce').min()
max_avg_wall = pd.to_numeric(df['totalMinWall'], errors='coerce').max()
min_od = pd.to_numeric(df['HotDiam'], errors='coerce').min()
max_od = pd.to_numeric(df['HotDiam'], errors='coerce').max()




# Min max summary

In [ ]:
# Create a dictionary
data = {
    'min_wall': [min_wall],
    'min_avg_wall': [min_avg_wall],
    'max_avg_wall': [max_avg_wall],
    'min_od': [min_od],
    'max_od': [max_od]
}

# Convert to DataFrame
summary_df = pd.DataFrame(data)

# Display the DataFrame
summary_df

,min_wall,min_avg_wall,max_avg_wall,min_od,max_od
0,5.091971,5.091971,14.905051,100.552212,198.688694


In [ ]:
# Identify missing data and dashes
missing_data = df.isna().sum()
missing_data

,0
Unnamed: 0,0
TimeStamp,0
HotDiam,20
totalMinWall,20


In [ ]:
# The Sum of dashes in each column
dashes = df.map(lambda x: x == '-').sum()
dashes

,0
Unnamed: 0,0
TimeStamp,0
HotDiam,0
totalMinWall,0


In [ ]:
# Identify the columns where dashes are
columns_with_dashes = [col for col in df.columns if (df[col] == '-').any()]

columns_with_dashes

[]

In [ ]:
# Identify the rows where dashes exist
rows_with_dashes = df[df.apply(lambda row: (row == '-').any(), axis=1)]
rows_with_dashes

,Unnamed: 0,TimeStamp,HotDiam,totalMinWall


In [ ]:
 #!pip install jupyter_bokeh

In [ ]:
# import packages to visual the data (control chart)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_time_control_chart(data):
    """
    Create a control chart with TimeStamp on x-axis and OD on y-axis

    Parameters:
    data (pd.DataFrame): Input dataframe with TimeStamp and OD columns
    """
    # Calculate control limits
    mean = pd.to_numeric(df['HotDiam'], errors='coerce').mean()
    std = pd.to_numeric(df['HotDiam'], errors='coerce').std()
    ucl = 180
    lcl = 120

    # Create figure
    fig = go.Figure()

    # Add main data line
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=data['HotDiam'],
                  mode='lines+markers',
                  name='OD',
                  line=dict(color='blue'))
    )

    # Add mean line
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[mean]*len(data),
                  mode='lines',
                  name='Mean',
                  line=dict(color='green', dash='dash'))
    )

    # Add UCL and LCL
    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[ucl]*len(data),
                  mode='lines',
                  name='UCL',
                  line=dict(color='red', dash='dash'))
    )

    fig.add_trace(
        go.Scatter(x=data['TimeStamp'], y=[lcl]*len(data),
                  mode='lines',
                  name='LCL',
                  line=dict(color='red', dash='dash'))
    )

    # Update layout
    fig.update_layout(
        title='Control Chart for OD over Time',
        xaxis_title='Time',
        yaxis_title='OD',
        height=600,
        showlegend=True,
        hovermode='x unified'
    )

    # Configure x-axis to handle dates better
    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    return fig

# Example usage:
"""
# Make sure TimeStamp is in datetime format
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])

# Create and show the chart
fig = create_time_control_chart(df)
fig.show()
"""

"\n# Make sure TimeStamp is in datetime format\ndf['TimeStamp'] = pd.to_datetime(df['TimeStamp'])\n\n# Create and show the chart\nfig = create_time_control_chart(df)\nfig.show()\n"

In [ ]:
fig = create_time_control_chart(df)
fig.show()

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def filter_by_timerange(df, start_time=None, end_time=None):
    """
    Filter DataFrame rows based on TimeStamp range

    Parameters:
    df (pd.DataFrame): Input dataframe with TimeStamp column
    start_time: Start time (string or datetime object)
    end_time: End time (string or datetime object)

    Returns:
    pd.DataFrame: Filtered dataframe
    """
    # Create a copy to avoid modifying original data
    filtered_df = df.copy()

    # Convert TimeStamp to datetime if it isn't already
    if not pd.api.types.is_datetime64_any_dtype(filtered_df['TimeStamp']):
        filtered_df['TimeStamp'] = pd.to_datetime(filtered_df['TimeStamp'])

    # Apply start time filter if provided
    if start_time is not None:
        if isinstance(start_time, str):
            start_time = pd.to_datetime(start_time)
        filtered_df = filtered_df[filtered_df['TimeStamp'] >= start_time]

    # Apply end time filter if provided
    if end_time is not None:
        if isinstance(end_time, str):
            end_time = pd.to_datetime(end_time)
        filtered_df = filtered_df[filtered_df['TimeStamp'] <= end_time]

    return filtered_df

def delete_time_period(df, start_time, end_time):
    """
    Delete rows between specified time range

    Parameters:
    df (pd.DataFrame): Input dataframe with TimeStamp column
    start_time: Start time to delete from (string or datetime object)
    end_time: End time to delete to (string or datetime object)

    Returns:
    pd.DataFrame: DataFrame with specified time period removed
    """
    # Create a copy to avoid modifying original data
    modified_df = df.copy()

    # Convert TimeStamp to datetime if it isn't already
    if not pd.api.types.is_datetime64_any_dtype(modified_df['TimeStamp']):
        modified_df['TimeStamp'] = pd.to_datetime(modified_df['TimeStamp'])

    # Convert input times to datetime if they're strings
    if isinstance(start_time, str):
        start_time = pd.to_datetime(start_time)
    if isinstance(end_time, str):
        end_time = pd.to_datetime(end_time)

    # Delete rows between start_time and end_time
    modified_df = modified_df[~((modified_df['TimeStamp'] >= start_time) &
                               (modified_df['TimeStamp'] <= end_time))]

    return modified_df



In [ ]:
# Example usage:

# Filter data for a specific time range
filtered_data = filter_by_timerange(
    df,
    start_time='2023-01-01 0:00:01',
    end_time='2023-01-01 0:00:09'
)

filtered_data

,Unnamed: 0,TimeStamp,HotDiam,totalMinWall
1,1,2023-01-01 00:00:01,195.071431,5.841400
2,2,2023-01-01 00:00:02,173.199394,6.616287
3,3,2023-01-01 00:00:03,159.865848,13.985542
4,4,2023-01-01 00:00:04,115.601864,11.064291
5,5,2023-01-01 00:00:05,115.599452,5.091971
6,6,2023-01-01 00:00:06,105.808361,6.014715
7,7,2023-01-01 00:00:07,186.617615,11.635018
8,8,2023-01-01 00:00:08,NaN,NaN
9,9,2023-01-01 00:00:09,170.807258,6.608081


In [ ]:
# Delete data for a specific time period
data_with_deletion = delete_time_period(
    df,
    start_time='2023-01-01 0:00:09',
    end_time='2023-01-01 03:00:00'
)
data_with_deletion


,Unnamed: 0,TimeStamp,HotDiam,totalMinWall
0,0,2023-01-01 00:00:00,137.454012,11.420316
1,1,2023-01-01 00:00:01,195.071431,5.841400
2,2,2023-01-01 00:00:02,173.199394,6.616287
3,3,2023-01-01 00:00:03,159.865848,13.985542
4,4,2023-01-01 00:00:04,115.601864,11.064291
5,5,2023-01-01 00:00:05,115.599452,5.091971
6,6,2023-01-01 00:00:06,105.808361,6.014715
7,7,2023-01-01 00:00:07,186.617615,11.635018
8,8,2023-01-01 00:00:08,NaN,NaN


In [ ]:

# # You can also use datetime objects
# from datetime import datetime
# start = datetime(2024, 1, 1, 8, 0)  # 2024-01-01 08:00:00
# end = datetime(2024, 1, 1, 16, 0)   # 2024-01-01 16:00:00
# filtered_data = filter_by_timerange(df, start_time=start, end_time=end)

In [ ]:
Control_chart_after_filter = create_time_control_chart(data_with_deletion)
Control_chart_after_filter.show()

In [ ]:
# # Save the DataFrame to an Excel file
data_with_deletion.to_excel('WO4573587.xlsx', index=True)

In [ ]:
gaz = pd.read_excel("/content/WO4573587.xlsx")

In [ ]:
gaz

,Unnamed: 0.1,Unnamed: 0,TimeStamp,HotDiam,totalMinWall
0,0,0,2023-01-01 00:00:00,137.454012,11.420316
1,1,1,2023-01-01 00:00:01,195.071431,5.841400
2,2,2,2023-01-01 00:00:02,173.199394,6.616287
3,3,3,2023-01-01 00:00:03,159.865848,13.985542
4,4,4,2023-01-01 00:00:04,115.601864,11.064291
5,5,5,2023-01-01 00:00:05,115.599452,5.091971
6,6,6,2023-01-01 00:00:06,105.808361,6.014715
7,7,7,2023-01-01 00:00:07,186.617615,11.635018
8,8,8,2023-01-01 00:00:08,NaN,NaN
